# Planar 4-Bar Linkage Kinematics: Analytical and Numerical Methods

This notebook demonstrates two approaches for solving the kinematic analysis of a planar 4-bar linkage:

- **Analytical Solution:** Uses the elimination method to solve for all possible configurations.
- **Numerical Solution:** Uses Newton's method (fsolve) to solve the kinematic equations for a given initial guess.

Both methods are implemented in Python and visualized. The notebook is designed for Google Colab and includes all code and figures needed to understand and reproduce the analysis.

<img src="fourbar_kinzelbook.png" alt="Four-bar linkage diagram" width="400" />

*Figure: Example configuration of the planar 4-bar linkage (from Kinzel book)*

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import fsolve

## Method 1: Analytical Solution

This method uses the elimination approach described in the lecture notes to solve the kinematic equations of the 4-bar linkage. It returns both possible solutions for each input crank angle.

In [ ]:
# Analytical solution for 4-bar linkage
r1 = 1
r2 = 2
r3 = 3.5
r4 = 4
th1 = 0
beta = np.arctan(1/2)
r6 = 2 / np.cos(beta)
th2_vals = np.deg2rad(np.arange(0, 361, 10))

th3_1 = np.zeros_like(th2_vals)
th3_2 = np.zeros_like(th2_vals)
th4_1 = np.zeros_like(th2_vals)
th4_2 = np.zeros_like(th2_vals)
posE_1 = np.zeros((len(th2_vals), 2))
posE_2 = np.zeros((len(th2_vals), 2))

for i, th2 in enumerate(th2_vals):
    A = 2 * r1 * r4 * np.cos(th1) - 2 * r2 * r4 * np.cos(th2)
    B = 2 * r1 * r4 * np.sin(th1) - 2 * r2 * r4 * np.sin(th2)
    C = r1**2 + r2**2 + r4**2 - r3**2 - 2 * r1 * r2 * np.cos(th1 - th2)
    # Both solutions for th4
    th4_1[i] = 2 * np.arctan((-B + np.sqrt(B**2 - C**2 + A**2)) / (C - A))
    th4_2[i] = 2 * np.arctan((-B - np.sqrt(B**2 - C**2 + A**2)) / (C - A))
    # Corresponding th3
    th3_1[i] = np.arctan2(
        r1 * np.sin(th1) + r4 * np.sin(th4_1[i]) - r2 * np.sin(th2),
        r1 * np.cos(th1) + r4 * np.cos(th4_1[i]) - r2 * np.cos(th2)
    )
    th3_2[i] = np.arctan2(
        r1 * np.sin(th1) + r4 * np.sin(th4_2[i]) - r2 * np.sin(th2),
        r1 * np.cos(th1) + r4 * np.cos(th4_2[i]) - r2 * np.cos(th2)
    )
    # Coupler point E for both solutions
    posA = r2 * np.array([np.cos(th2), np.sin(th2)])
    posE_1[i] = posA + r6 * np.array([np.cos(th3_1[i] + beta), np.sin(th3_1[i] + beta)])
    posE_2[i] = posA + r6 * np.array([np.cos(th3_2[i] + beta), np.sin(th3_2[i] + beta)])

plt.figure()
plt.plot(np.rad2deg(th2_vals), np.rad2deg(th3_1), label='th3 (solution 1)')
plt.plot(np.rad2deg(th2_vals), np.rad2deg(th3_2), label='th3 (solution 2)')
plt.plot(np.rad2deg(th2_vals), np.rad2deg(th4_1), label='th4 (solution 1)')
plt.plot(np.rad2deg(th2_vals), np.rad2deg(th4_2), label='th4 (solution 2)')
plt.xlabel('Crank Angle (deg)')
plt.ylabel('Angle (deg)')
plt.title('Coupler and Output Angles vs. Input Crank Angle (Analytical)')
plt.grid(True)
plt.legend()
plt.show()

plt.figure()
plt.plot(posE_1[:,0], posE_1[:,1], 'r-', label='Coupler E Trajectory (sol 1)')
plt.plot(posE_2[:,0], posE_2[:,1], 'b--', label='Coupler E Trajectory (sol 2)')
plt.xlabel('X Position')
plt.ylabel('Y Position')
plt.title('Trajectory of Coupler Point E (Analytical)')
plt.axis('equal')
plt.grid(True)
plt.legend()
plt.show()

## Method 2: Numerical Solution (Newton's Method)

This method uses Newton's iteration (fsolve) to solve the kinematic equations of the 4-bar linkage. It returns one solution for each input crank angle, depending on the initial guess.

In [ ]:
# Numerical solution for 4-bar linkage using Newton's method
r1 = 1
r2 = 2
r3 = 3.5
r4 = 4
th1 = 0
beta = np.arctan(1/2)
r6 = 2 / np.cos(beta)
th2list = np.deg2rad(np.arange(0, 361, 1))
n = len(th2list)
th3list = np.zeros(n)
th4list = np.zeros(n)
pElist = np.zeros((n, 2))
OB = np.array([1, 0])

def vecloopeq(x, th1, th2, r1, r2, r3, r4):
    th3, th4 = x
    eq1 = r2 * np.cos(th2) + r3 * np.cos(th3) - r4 * np.cos(th4) - r1 * np.cos(th1)
    eq2 = r2 * np.sin(th2) + r3 * np.sin(th3) - r4 * np.sin(th4) - r1 * np.sin(th1)
    return [eq1, eq2]

x0 = [-np.pi/3, -np.pi/3]
for i, th2 in enumerate(th2list):
    f = lambda x: vecloopeq(x, th1, th2, r1, r2, r3, r4)
    solth3th4 = fsolve(f, x0)
    th3, th4 = solth3th4
    th3list[i] = th3
    th4list[i] = th4
    x0 = solth3th4  # use previous solution as next initial guess
    pA = r2 * np.array([np.cos(th2), np.sin(th2)])
    pE = pA + r6 * np.array([np.cos(th3 + beta), np.sin(th3 + beta)])
    pElist[i] = pE

plt.figure()
plt.plot(np.rad2deg(th2list), np.rad2deg(th3list), label='Coupler Angle th3')
plt.plot(np.rad2deg(th2list), np.rad2deg(th4list), label='Output Angle th4')
plt.xlabel('Crank Angle (deg)')
plt.ylabel('Angle (deg)')
plt.title('Coupler and Output Angles vs. Input Crank Angle (Numerical)')
plt.grid(True)
plt.legend()
plt.show()

plt.figure()
plt.plot(pElist[:,0], pElist[:,1], 'r-', label='Coupler Point E Trajectory')
plt.xlabel('X Position')
plt.ylabel('Y Position')
plt.title('Trajectory of Coupler Point E (Numerical)')
plt.axis('equal')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
# Animation for Analytical Method (solution 1)
import matplotlib.animation as animation
from IPython.display import HTML

fig, ax = plt.subplots(figsize=(6,6))
ax.set_xlim(-5, 5)
ax.set_ylim(-5, 5)
ax.set_title('4-Bar Linkage Animation (Analytical, Solution 1)')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.grid(True)
linkage_line, = ax.plot([], [], 'o-', label='Linkage')
traj_line, = ax.plot([], [], 'r--', label='Coupler E Trajectory')
ax.legend()

def animate_analytical(i):
    posOA = np.array([0, 0])
    posA = r2 * np.array([np.cos(th2_vals[i]), np.sin(th2_vals[i])])
    posE = posE_1[i]
    posB = posA + r3 * np.array([np.cos(th3_1[i]), np.sin(th3_1[i])])
    posOB = np.array([1, 0])
    linkage = np.vstack([posOA, posA, posE, posB, posA, posB, posOB])
    linkage_line.set_data(linkage[:,0], linkage[:,1])
    traj_line.set_data(posE_1[:i+1,0], posE_1[:i+1,1])
    return linkage_line, traj_line

ani = animation.FuncAnimation(fig, animate_analytical, frames=len(th2_vals), interval=50, blit=True)
HTML(ani.to_jshtml())

In [ ]:
# Animation for Numerical Method
fig, ax = plt.subplots(figsize=(6,6))
ax.set_xlim(-5, 5)
ax.set_ylim(-5, 5)
ax.set_title('4-Bar Linkage Animation (Numerical)')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.grid(True)
linkage_line, = ax.plot([], [], 'o-', label='Linkage')
traj_line, = ax.plot([], [], 'r--', label='Coupler E Trajectory')
ax.legend()

def animate_numerical(i):
    posOA = np.array([0, 0])
    posA = r2 * np.array([np.cos(th2list[i]), np.sin(th2list[i])])
    posE = pElist[i]
    posB = posA + r3 * np.array([np.cos(th3list[i]), np.sin(th3list[i])])
    posOB = np.array([1, 0])
    linkage = np.vstack([posOA, posA, posE, posB, posA, posB, posOB])
    linkage_line.set_data(linkage[:,0], linkage[:,1])
    traj_line.set_data(pElist[:i+1,0], pElist[:i+1,1])
    return linkage_line, traj_line

ani = animation.FuncAnimation(fig, animate_numerical, frames=len(th2list), interval=50, blit=True)
HTML(ani.to_jshtml())